In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env

m = wt.watertap_setup(dynamic=False)


wt.case_study_trains.train = {"case_study": "carlsbad",
                             "reference": "nawi",
                             "scenario": "baseline"}


wt.case_study_trains.source_water = {"case_study": "carlsbad", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": 'seawater'}

m = wt.case_study_trains.get_case_study(m=m) # flow is set as case study flow unless defined.


cherokee

------- Adding Unit Processes -------
cooling_tower
assuming cycles of concentration is: 5.0
sedimentation
settling_pond
treated_storage_a
onsite_storage
chlorination
media_filtration
anti_scalant_addition
reverse_osmosis
brine_ro
treated_storage_b
industrial
discharge
-------------------------------------

adding source
adding source
adding source

Connecting unit processes...


In [2]:
wt.display.show_train2(model_name=m)

In [3]:
# RUN MODEL with optimal ro so that the model solves. Then runs again with set pressure. 

In [4]:
# # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("Unfixing feed presure and area for", key, '...\n')
        
        
wt.run_water_tap(m=m, objective=True, skip_small=False)

# prints RO results
print("\nOptimal RO area and pressures:")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(f'\tFeed pressure for {key}: {round(getattr(m.fs, key).feed.pressure[0](), 2)} bar')
        print(f'\tMembrane area for {key}: {round(getattr(m.fs, key).membrane_area[0](), 2)} m2')

# RESET PRESSURE TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        if "feed_pressure" in m.fs.pfd_dict[key]["Parameter"]:
            getattr(m.fs, key).feed.pressure.fix(m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])
            print(f'\nSetting feed presure for {key} to {m.fs.pfd_dict[key]["Parameter"]["feed_pressure"]} bar (user input)...\n')

wt.run_water_tap(m=m, objective=True, print_model_results="summary")

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(f'\tFeed pressure for {key}: {round(getattr(m.fs, key).feed.pressure[0](), 2)} bar')
        print(f'\tMembrane area: for {key}: {round(getattr(m.fs, key).membrane_area[0](), 2)} m2')

Unfixing feed presure and area for reverse_osmosis ...

Unfixing feed presure and area for brine_ro ...

Flow is relatively small (< 1 m3/s). Running model with larger dummy flows to initialize...

----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Model finished running to initialize conditions. Now running with actual flow...

----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solver returned an infeasible solution...
Running again with updated initial conditions --- attempt 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------

Optimal RO area and pressures:
	Feed pressure for reverse_osmosis: 17.61 bar
	Membrane area for reverse_osmosis: 1380.67 m2
	Feed pressure for brine_ro: 14.24 bar
	Membrane area for brine_ro: 10.0 m2

Setting feed pr

In [5]:
# set cap utilization factor
m.fs.costing_param.plant_cap_utilization = 1

In [6]:
wt.run_water_tap(m=m, objective=True, print_model_results="summary")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


COOLING TOWER:

	total cap investment: 0.0
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.0

SEDIMENTATION:

	total cap investment: 0.17571
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00234

SETTLING POND:

	total cap investment: 0.17519
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00233

TREATED STORAGE A:

	total cap investment: 0.2275
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00299

ONSITE STORAGE:

	total cap investment: 0.79657
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.01053

CHLORINATION:

	total cap investment: 1.69889
	cat and chem cost: 0.00099
	electricity cost: 0.0
	total fixed op cost: 0.02262

MEDIA FILTRATION:

In [7]:
# RESET AREA BASED ON RO RECOVERY
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).membrane_area.fix(getattr(m.fs, key).membrane_area[0]())

In [8]:
# Readjust recovery constraint and deactivate objective constraint
m.fs.objective_function.deactivate() # deactivate LCOW objective function

In [9]:
# NOW RUN AS SIMULATION
wt.run_water_tap(m=m, objective=False, print_model_results="summary")

----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


COOLING TOWER:

	total cap investment: 0.0
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.0

SEDIMENTATION:

	total cap investment: 0.17571
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00234

SETTLING POND:

	total cap investment: 0.17519
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00233

TREATED STORAGE A:

	total cap investment: 0.2275
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00299

ONSITE STORAGE:

	total cap investment: 0.79657
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.01053

CHLORINATION:

	total cap investment: 1.69889
	cat and chem cost: 0.00099
	electricity cost: 0.0
	total fixed op cost: 0.02262

MEDIA FILTRATION:

In [10]:
# creates csv in results folder with the name: *case_study*_*scenario*.csv
# In this case, save the final baseline result.

df = wt.get_results_table(m=m, case_study=wt.case_study_trains.source_water["case_study"], 
                                scenario=wt.case_study_trains.source_water["scenario"])

In [11]:
###### SENSITIVITY ANALYSES 

In [12]:
# Find bounds for RO area:
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        
        stash_value = value(getattr(m.fs, key).membrane_area[0])
        getattr(m.fs, key).membrane_area.unfix()
        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).membrane_area[0] * 1, sense=env.minimize)
        wt.run_water_tap(m = m, objective=False)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Minimum -->", getattr(m.fs, key).membrane_area[0]())

        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).membrane_area[0], sense=env.maximize)
        wt.run_water_tap(m = m, objective=False)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Maximum -->", getattr(m.fs, key).membrane_area[0]())

        getattr(m.fs, key).membrane_area.fix(stash_value)


----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.6393989686739137
reverse_osmosis Minimum --> 10.0
----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.43694548656829724
reverse_osmosis Maximum --> 752.8198908596981
----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.4369455239631826
brine_ro Minimum --> 10.0
----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.4369454865774635
br

In [13]:
# Find bounds for RO pessure:
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        
        stash_value = value(getattr(m.fs, key).feed.pressure[0])
        getattr(m.fs, key).feed.pressure.unfix()
        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).feed.pressure[0], sense=env.minimize)
        wt.run_water_tap(m = m, objective=False)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Minimum -->", getattr(m.fs, key).feed.pressure[0]())

        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).feed.pressure[0], sense=env.maximize)
        wt.run_water_tap(m = m, objective=False)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Maximum -->", getattr(m.fs, key).feed.pressure[0]())

        getattr(m.fs, key).feed.pressure.fix(stash_value)

----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.6400539637737545
reverse_osmosis Minimum --> 4.000214288220219
----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.4369454866043741
reverse_osmosis Maximum --> 30.000000026984296
----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.43824258983646036
brine_ro Minimum --> 4.000214288220376
----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCO

In [14]:
m.fs.objective_function1.deactivate()

In [15]:
# run to rest before sensitivity
wt.run_water_tap(m = m, objective=False)
print("LCOW -->", m.fs.costing.LCOW())

----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.43694548671369604
